In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#%%
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.keras import optimizers
from tensorflow.python.keras import activations
from tensorflow.python.keras import callbacks
from tensorflow.python.keras import initializers
from tensorflow.python.keras import backend
from tensorflow.python.keras import metrics
from tensorflow import keras
import tensorflow as tf
import keras_tuner as kt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import copy

In [ ]:
train_x_rc = np.load('../input/ventilator-traindata/train_x_rc.npy')
train_x_other = np.load('../input/ventilator-traindata/train_x_other.npy')
targets = np.load('../input/ventilator-traindata/target.npy')
print(train_x_other.shape)
print(train_x_rc.shape)
print(targets.shape)
train_x_ = np.concatenate([train_x_other, train_x_rc], axis=-1)
del train_x_rc

In [ ]:
class ExpandTileLayer(layers.Layer):
    def __init__(self):
        super(ExpandTileLayer, self).__init__()
    def call(self, inputs, *args, **kwargs):
        return backend.tile(backend.expand_dims(inputs, axis=-2), (1, 80, 1))

In [ ]:
def build_model():
    rc_input = keras.Input(shape=(15, ),
                           dtype='int32',
                           name='rc_input_layer')
    other_x_input = keras.Input(shape=(80, train_x_other.shape[-1]),
                                dtype='float32',
                                name='other_x_input')
    rc_embedding_layer = layers.Dense(units=16,
                                      use_bias=False, name='rc_embed_layer')
    rc_embedding = rc_embedding_layer(rc_input)

    # 维度拓展
    rc_embedding = ExpandTileLayer()(rc_embedding)

    x_input = layers.Concatenate(axis=-1)([other_x_input, rc_embedding])
    conv1d_1 = layers.Conv1D(filters=256,
                             kernel_size=5,
                             padding='same',
                             use_bias=False)(x_input)
    conv1d_1 = layers.Activation(activations.selu)(conv1d_1)
    conv1d_2 = layers.Conv1D(filters=128, use_bias=False,
                             kernel_size=5, padding='same')(conv1d_1)
    conv1d_output = layers.Activation(activations.selu)(conv1d_2)
    ox = layers.Bidirectional(layers.LSTM(units=512,
                                          return_sequences=True,
                                          kernel_initializer=initializers.initializers_v2.GlorotUniform()),
                              merge_mode='concat')(conv1d_output)

    ox = layers.Bidirectional(layers.LSTM(units=384,
                                          return_sequences=True,
                                          kernel_initializer=initializers.initializers_v2.GlorotUniform()),
                              merge_mode='concat')(ox)
    ox = layers.Bidirectional(layers.LSTM(units=256,
                                          return_sequences=True,
                                          kernel_initializer=initializers.initializers_v2.GlorotUniform()),
                              merge_mode='concat')(ox)
    ox = layers.Dense(units=128,
                      kernel_initializer=initializers.initializers_v2.GlorotUniform())(ox)
    lstm_output = layers.ELU()(ox)

    output = layers.Concatenate(axis=-1)([lstm_output, conv1d_output])
    output = layers.Dense(units=256,
                          activation=activations.selu,
                          kernel_initializer=initializers.initializers_v2.GlorotUniform())(output)
    output = layers.Dense(units=1,
                          kernel_initializer=initializers.initializers_v2.GlorotUniform())(output)

    my_model = models.Model(inputs=[rc_input, other_x_input], outputs=[output])
    return my_model
model = build_model()
keras.utils.plot_model(model, './model1.png')

In [ ]:
# def build_model():
#     rc_input = keras.Input(shape=(15, ),
#                            dtype='int32',
#                            name='rc_input_layer')
#     other_x_input = keras.Input(shape=(80, train_x_other.shape[-1]),
#                                 dtype='float32',
#                                 name='other_x_input')
#     rc_embedding_layer = layers.Dense(units=10,
#                                       use_bias=False, 
#                                       activation=activations.selu,
#                                       name='rc_embed_layer')
#     rc_embedding = rc_embedding_layer(rc_input)

#     # 维度拓展
#     rc_embedding = ExpandTileLayer()(rc_embedding)

#     x_input = layers.Concatenate(axis=-1)([other_x_input, rc_embedding])
#     conv1d_1 = layers.Conv1D(filters=256,
#                              kernel_size=5,
#                              padding='same')(x_input)
#     conv1d_1 = layers.Activation(activations.relu)(conv1d_1)
#     conv1d_1 = layers.BatchNormalization()(conv1d_1)
#     conv1d_2 = layers.Conv1D(filters=192, 
#                              kernel_size=5,
#                              padding='same')(conv1d_1)
#     conv1d_output = layers.Activation(activations.relu)(conv1d_2)
#     conv1d_output = layers.BatchNormalization()(conv1d_output)
#     the_feature = layers.Concatenate()([conv1d_output, x_input])
#     ox_1 = layers.Bidirectional(layers.LSTM(units=672,
#                                           return_sequences=True,
#                                           kernel_initializer=initializers.initializers_v2.GlorotUniform()),
#                               merge_mode='concat')(the_feature)
#     ox_2 = layers.Bidirectional(layers.LSTM(units=512,
#                                           return_sequences=True,
#                                           kernel_initializer=initializers.initializers_v2.GlorotUniform()),
#                               merge_mode='concat')(ox_1)
#     ox = layers.Bidirectional(layers.LSTM(units=384,
#                                           return_sequences=True,
#                                           kernel_initializer=initializers.initializers_v2.GlorotUniform()),
#                               merge_mode='concat')(ox_2)
#     ox = layers.Bidirectional(layers.GRU(units=192,
#                     return_sequences=True))(ox)
#     output = layers.Concatenate(axis=-1)([ox, conv1d_output])
#     output = layers.Dense(units=128,
#                           activation=activations.selu,
#                           kernel_initializer=initializers.initializers_v2.GlorotUniform())(output)
#     output = layers.Dense(units=1,
#                           kernel_initializer=initializers.initializers_v2.GlorotUniform())(output)

#     my_model = models.Model(inputs=[rc_input, other_x_input], outputs=[output])
    
#     return my_model

In [ ]:

# def build_model():
#     rc_input = keras.Input(shape=(15, ),
#                            dtype='int32',
#                            name='rc_input_layer')
#     other_x_input = keras.Input(shape=(80, train_x_other.shape[-1]),
#                                 dtype='float32',
#                                 name='other_x_input')
#     rc_embed = layers.Dense(units=10, use_bias=False, name='rc_embed_layer')(rc_input)
#     rc_embedding = ExpandTileLayer()(rc_embed)
#     x_input = layers.Concatenate(axis=-1)([other_x_input, rc_embedding])
#     conv_fea = layers.Conv1D(kernel_initializer=initializers.initializers_v2.LecunNormal(),
#                              filters=128, kernel_size=5, padding='same')(x_input)
#     conv_fea = layers.Activation(activations.selu)(conv_fea)

#     x1 = layers.Bidirectional(layers.LSTM(units=640, return_sequences=True))(x_input)
#     x2 = layers.Bidirectional(layers.LSTM(units=512, return_sequences=True))(x1)
#     x3 = layers.Bidirectional(layers.LSTM(units=384, return_sequences=True))(x2)
#     x4 = layers.Bidirectional(layers.LSTM(units=256, return_sequences=True))(x3)
    
#     xc = layers.Concatenate()([x2, conv_fea])
#     z2 = layers.Bidirectional(layers.GRU(units=384, return_sequences=True))(x2)

#     z31 = layers.Multiply()([x3, z2])
#     z31 = layers.BatchNormalization()(z31)
#     z3 = layers.Bidirectional(layers.GRU(units=256, return_sequences=True))(z31)

#     z41 = layers.Multiply()([x4, z3])
#     z41 = layers.BatchNormalization()(z41)
#     z4 = layers.Bidirectional(layers.GRU(units=128, return_sequences=True))(z41)

#     x = layers.Concatenate(axis=2)([x4, z2, z3, z4, conv_fea])

#     x = layers.Dense(units=160, activation='selu')(x)

#     x_output = layers.Dense(units=1, use_bias=False)(x)

#     my_model = models.Model(inputs=[rc_input, other_x_input], outputs=[x_output])
#     return my_model




In [ ]:
# def build_model():
#     rc_input = keras.Input(shape=(15,),
#                            dtype='int32',
#                            name='rc_input_layer')
#     other_x_input = keras.Input(shape=(80, train_x_other.shape[-1]),
#                                 dtype='float32',
#                                 name='other_x_input')
#     rc_embedding_layer = layers.Dense(units=10,
#                                       use_bias=False, name='rc_embed_layer')
#     rc_embedding = rc_embedding_layer(rc_input)

#     # 维度拓展
#     rc_embedding = ExpandTileLayer()(rc_embedding)

#     x_input = layers.Concatenate(axis=-1)([other_x_input, rc_embedding])
#     conv1d_1 = layers.Conv1D(filters=256,
#                              kernel_size=5,
#                              padding='same',
#                              use_bias=False)(x_input)
#     conv1d_1 = layers.Activation(activations.selu)(conv1d_1)
#     conv1d_2 = layers.Conv1D(filters=128,
#                              kernel_size=5,
#                              padding='same')(conv1d_1)
#     conv1d_2 = layers.BatchNormalization()(conv1d_2)
#     conv1d_output = layers.Activation(activations.selu)(conv1d_2)
#     ox = layers.Bidirectional(layers.LSTM(units=784,
#                                           return_sequences=True,
#                                           kernel_initializer=initializers.initializers_v2.GlorotUniform()),
#                               merge_mode='concat')(conv1d_output)
#     ox_1 = layers.Bidirectional(layers.LSTM(units=512,
#                                             return_sequences=True,
#                                             kernel_initializer=initializers.initializers_v2.GlorotUniform()),
#                                 merge_mode='concat')(ox)
#     ox_2 = layers.Bidirectional(layers.LSTM(units=384,
#                                             return_sequences=True,
#                                             kernel_initializer=initializers.initializers_v2.GlorotUniform()),
#                                 merge_mode='concat')(ox_1)
#     ox = layers.Bidirectional(layers.LSTM(units=128, return_sequences=True),
#                              merge_mode='concat')(ox_2)
    
# #     z3 = layers.Bidirectional(layers.GRU(units=384, return_sequences=True),
# #                              merge_mode='mul')(ox_1)
# #     z4 = layers.Multiply()([ox_2, z3])
# #     z4 = layers.BatchNormalization()(z4)
# #     z4 = layers.Bidirectional(layers.LSTM(units=128, return_sequences=True),
# #                              merge_mode='concat')(z4)
#     feature_output = layers.Concatenate(axis=-1)([ox, x_input])
#     output = layers.Dense(units=128, 
#                           activation=activations.selu,
#                           kernel_initializer=initializers.initializers_v2.GlorotUniform())(feature_output)
# #     output = layers.PReLU()(output)
#     output = layers.Dense(units=1,
#                           kernel_initializer=initializers.initializers_v2.GlorotUniform())(output)

#     my_model = models.Model(inputs=[rc_input, other_x_input], outputs=[output])
#     return my_model


# GPU训练

In [ ]:
# 6折交叉验证
kf = KFold(n_splits=7, shuffle=True, random_state=1143)
for fold, (train_ind, test_ind) in enumerate(kf.split(train_x_, targets)):
    print('-------------------------------------------------------------------------------------------------')
    train_x, test_x = train_x_[train_ind], train_x_[test_ind]
    train_y, test_y = targets[train_ind], targets[test_ind]
    train_x_rc = train_x[:, :, -15:][:, 0, :]
    train_x_ox = train_x[:, :, :-15]
    test_x_rc = test_x[:, :, -15:][:, 0, :]
    test_x_ox = test_x[:, :, :-15]
    train_y = train_y.reshape((-1, 80, 1))
    test_y = test_y.reshape((-1, 80, 1))
    
    # 样本加权，区分呼气阶段和吸气阶段
    y_weight = train_x_ox[:, :, 2] + 2
    y_weight = y_weight.reshape((-1, 80, 1))
    y_weight[y_weight == 2] = 0.9
    test_weight = test_x_ox[:, :, 2] + 2
    test_weight[test_weight == 2] = 0
    test_weight = test_weight.reshape((-1, 80, 1))

    my_model = build_model()
    if fold == 0:
        print(my_model.summary())

    my_callbacks = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.65, patience=10, verbose=1),
                    callbacks.EarlyStopping(monitor='val_loss', patience=29, verbose=1, mode='min',
                                            restore_best_weights=True)]
    my_model.compile(loss=losses.MeanAbsoluteError(),
                     optimizer=optimizers.Adam(clipnorm=0.9),
                     sample_weight_mode="temporal")
    my_model.fit(x=[train_x_rc, train_x_ox], y=train_y,
                 sample_weight=y_weight,
                 validation_data=([test_x_rc, test_x_ox], test_y, test_weight),
                 epochs=300,
                 batch_size=512,
                 validation_batch_size=4096,
                 callbacks=my_callbacks,
                 verbose=1)
    my_model.save_weights(f'./model_{fold}.h5')

# TPU训练

In [ ]:

# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# # instantiating the model in the strategy scope creates the model on the TPU
# with tpu_strategy.scope():
#     kf = KFold(n_splits=6, shuffle=True, random_state=1143)
#     for fold, (train_ind, test_ind) in enumerate(kf.split(train_x_, targets)):
#         print('-------------------------------------------------------------------------------------------------')
#         train_x, test_x = train_x_[train_ind], train_x_[test_ind]
#         train_y, test_y = targets[train_ind], targets[test_ind]
#         train_x_rc = train_x[:, :, -15:][:, 0, :]
#         train_x_ox = train_x[:, :, :-15]
#         test_x_rc = test_x[:, :, -15:][:, 0, :]
#         test_x_ox = test_x[:, :, :-15]
#         train_y = train_y.reshape((-1, 80, 1))
#         test_y = test_y.reshape((-1, 80, 1))

#         y_weight = train_x_ox[:, :, 2] + 2
#         y_weight = y_weight.reshape((-1, 80, 1))
#         y_weight[y_weight == 2] = 0.9
#         test_weight = test_x_ox[:, :, 2] + 2
#         test_weight[test_weight == 2] = 0
#         test_weight = test_weight.reshape((-1, 80, 1))

#         my_model = build_model()
#         if fold == 0:
#             print(my_model.summary())
            
#         my_callbacks = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.65, patience=10, verbose=1),
#                         callbacks.EarlyStopping(monitor='val_loss', patience=29, verbose=1, mode='min',
#                                                 restore_best_weights=True)]
#         my_model.compile(loss=losses.MeanAbsoluteError(),
#                          optimizer=optimizers.Adam(clipnorm=0.9),
#                          sample_weight_mode="temporal")
#         my_model.fit(x=[train_x_rc, train_x_ox], y=train_y,
#                      sample_weight=y_weight,
#                      validation_data=([test_x_rc, test_x_ox], test_y, test_weight),
#                      epochs=300,
#                      batch_size=1024,
#                      validation_batch_size=5120,
#                      callbacks=my_callbacks,
#                      verbose=1)
#         save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
#         my_model.save_weights(f'./model_final_{fold}.h5', options=save_locally)

# 生成提交文件

In [ ]:
test_x_other = np.load('../input/ventilator-traindata/test_x_other.npy')
test_x_rc = np.load('../input/ventilator-traindata/test_x_rc.npy')

In [ ]:
pre_pressure = []
for fold in range(7):
    my_model.load_weights(f'./model_{fold}.h5')
    pre_y_ = my_model.predict(x=[test_x_rc, test_x_other], batch_size=512)
    pre_y = np.array(pre_y_).flatten()
    pre_pressure.append(pre_y)

In [ ]:
pressure_step = 0.07030248641967773
p_min = -1.7551400036622216
p_max = 64.82099173863328
sub_medclip = np.median(np.vstack(pre_pressure), axis=0)
sub_medclip = np.round((sub_medclip - p_min) / pressure_step) * pressure_step + p_min
sub_medclip = np.vstack([np.arange(1, sub_medclip.shape[0] + 1), sub_medclip])
sub_medclip = np.transpose(sub_medclip, axes=(1, 0))
sub_medclip = pd.DataFrame(sub_medclip, columns=['id', 'pressure'])
sub_medclip['id'] = sub_medclip['id'].astype('int32')
sub_medclip['pressure'] = sub_medclip['pressure'].astype('float32')
sub_medclip.to_csv('./submission.csv', index=False)